<h1>Práctica Final de Spark - 2. Aplicación del modelo en streaming</h1>
<p/>
<table border>
<tr><td>Alumno</td><td>email</td><td>Teléfono</td></ur>
<tr><td>José María Álvarez</td><td>josemaria.alvarezfernandez@elcorteingles.es</td><td>+34 682 780 953</td></ur>
<tr><td>Adolfo González</td><td>adolfo.gonzalez@elcorteingles.es</td><td>+34 609 964 414</td></ur>
<tr><td>César Colado</td><td>cesar.colado@elcorteingles.es</td><td>+34 661 415 555</td></ur>
</table>

Para la carga el entrenamiento se usó un dataset de vinos tintos. Ahora para la carga en streaming se usará el de vinos blancos, y se intentará predecir con los clusters que se sacaron para los vinos tintos. Si lo que dicen los sumillers es cierto, los tintos y los blancos son vinos muy diferentes, pero hemos querido probar esto como una manera más de intentar hacer esto divertido. Evidementemente no esperamos grandes resultados, pero si ejemplificar que hemos entendido la materia.

La alternativa sería hacerlo con los mismo datos de entrenamiento o un subconjunto del dataset que hemos utilizado como generador del modelo. No obstante, como no somos capaces de hacer una evaluación objetiva, no sabríamos si el cluster asignado al ejemplar evaluado es bueno o malo. 

Se ha creado un programa en python (python_streaming.py) que carga el fichero desde Internet (http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv), y lo va introduciendo en ficheros en un directorio, cada fila en un fichero, y crea uno cada segundo.

Lo primero que hacemos es cargar el modelo anteriormente salvado.

In [1]:
from pyspark.ml.clustering import KMeansModel
model = KMeansModel.load("wine_kmeans_model")

Después lo que vamos a hacer es usar Spark Structured Streaming (en vez de Spark Streaming y DStreams) para poder procesar los datos con un schema definido, y poder utilizar dichos datos para generar una tabla en memoria de donde iremos leyendo los datos. Se ha hecho con una tabla debido a que el output de consola lo saca por la consola estandar de la sesión de pyspark, y no la consola del notebook, con lo cual los resultados que se obtenian no se veían aquí. Se genera el esquema a importar, y se cargan todos los ficheros csv del directorio, a medida que el script los va generando, y se van haciendo predicciones sobre los datos en streaming, añadiéndose a la tabla a medida que se generan (el output mode es append).

In [2]:
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler
schema = StructType([
    StructField("fixed acidity", DoubleType()),
    StructField("volatile acidity", DoubleType()), 
    StructField("citric acid", DoubleType()), 
    StructField("residual sugar", DoubleType()), 
    StructField("chlorides", DoubleType()), 
    StructField("free sulfur dioxide", DoubleType()), 
    StructField("total sulfur dioxide", DoubleType()), 
    StructField("density", DoubleType()), 
    StructField("pH", DoubleType()), 
    StructField("sulphates", DoubleType()), 
    StructField("alcohol", DoubleType()), 
    StructField("quality", IntegerType()),
    StructField("id", StringType())])
wineDF = spark.readStream.option("sep", ";").option("header", "false") \
    .schema(schema) \
    .format("csv").load("./data")
vecAssembler = VectorAssembler(inputCols=["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density", "pH", "sulphates", "alcohol"], outputCol="features")
wineDF_kmeans = vecAssembler.transform(wineDF).select('id', 'features')
predictions = model.transform(wineDF_kmeans).select('id', 'prediction')
query = predictions \
    .writeStream \
    .queryName("predictions_ml") \
    .outputMode("append") \
    .format("memory") \
    .start()

Después, para poder ver los resultados, hacemos durante 20 segundos una query a la tabla para ver como va creciendo con los ficheros procesados.

In [3]:
import time
i = 0
while (i < 20):
    time.sleep(1.0)
    spark.sql("select * from predictions_ml").show()
    i+=1

+-----+----------+
|   id|prediction|
+-----+----------+
|wine4|         5|
|wine3|        10|
|wine2|         5|
|wine1|         5|
|wine5|         5|
|wine6|        10|
|wine7|         5|
|wine8|         5|
+-----+----------+

+------+----------+
|    id|prediction|
+------+----------+
| wine4|         5|
| wine3|        10|
| wine2|         5|
| wine1|         5|
| wine5|         5|
| wine6|        10|
| wine7|         5|
| wine8|         5|
| wine9|         5|
|wine10|         5|
+------+----------+

+------+----------+
|    id|prediction|
+------+----------+
| wine4|         5|
| wine3|        10|
| wine2|         5|
| wine1|         5|
| wine5|         5|
| wine6|        10|
| wine7|         5|
| wine8|         5|
| wine9|         5|
|wine10|         5|
|wine11|         7|
+------+----------+

+------+----------+
|    id|prediction|
+------+----------+
| wine4|         5|
| wine3|        10|
| wine2|         5|
| wine1|         5|
| wine5|         5|
| wine6|        10|
| wine7| 